In [18]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

season = 2021
SITE_URL = "https://www.pro-football-reference.com"
url = SITE_URL + "/years/" + str(season) + "/games.htm"
req = requests.get(url)
soup = BeautifulSoup(req.text, "lxml")
calendar_div = soup.find('table', id='games')
game_urls = []
game_ids = []
for link in calendar_div.find_all("a"):
    if link.text == "boxscore" or link.text == "preview":
        game_url = link.get("href")
        game_urls.append(SITE_URL + game_url)
        game_id = re.findall("\d{9}\D{3}", game_url)[0]
        game_ids.append(game_id)

calendar = pd.read_html(str(calendar_div))[0]
calendar = calendar[calendar['Day'].isin('Mon Tue Wed Thu Fri Sat Sun'.split())]
calendar = calendar.copy().reset_index(drop=True)
calendar.columns = "week day date time away at home url ptsw ptsl ydsw tow ydsl tol".split()
calendar['has_data'] = [True if row == 'boxscore' else False for row in calendar['url']]
for index, row in calendar.iterrows():
    if all([row['has_data'], row['at'] != '@']):
        calendar.at[index, 'away'] = row['home']
        calendar.at[index, 'home'] = row['away']

calendar['url'] = game_urls
calendar['id'] = game_ids

calendar = calendar['id week day date time away home url'.split()].copy()

for i, row in calendar.iterrows():
    print(row.to_dict())

{'id': '202109090tam', 'week': '1', 'day': 'Thu', 'date': '2021-09-09', 'time': '8:20PM', 'away': 'Dallas Cowboys', 'home': 'Tampa Bay Buccaneers', 'url': 'https://www.pro-football-reference.com/boxscores/202109090tam.htm'}
{'id': '202109120atl', 'week': '1', 'day': 'Sun', 'date': '2021-09-12', 'time': '1:00PM', 'away': 'Philadelphia Eagles', 'home': 'Atlanta Falcons', 'url': 'https://www.pro-football-reference.com/boxscores/202109120atl.htm'}
{'id': '202109120buf', 'week': '1', 'day': 'Sun', 'date': '2021-09-12', 'time': '1:00PM', 'away': 'Pittsburgh Steelers', 'home': 'Buffalo Bills', 'url': 'https://www.pro-football-reference.com/boxscores/202109120buf.htm'}
{'id': '202109120car', 'week': '1', 'day': 'Sun', 'date': '2021-09-12', 'time': '1:00PM', 'away': 'New York Jets', 'home': 'Carolina Panthers', 'url': 'https://www.pro-football-reference.com/boxscores/202109120car.htm'}
{'id': '202109120cin', 'week': '1', 'day': 'Sun', 'date': '2021-09-12', 'time': '1:00PM', 'away': 'Minnesota V

In [2]:
calendar.to_json(index)

ValueError: Invalid file path or buffer object type: <class 'int'>